In [1]:
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [3]:
import pygame
from random import randint
from copy import deepcopy
from PIL import Image
import numpy as np

# Initialize constants
RES = WIDTH, HEIGHT = 1500, 900
TILE = 30
W, H = WIDTH // TILE, HEIGHT // TILE
FPS = 6

pygame.init()
surface = pygame.display.set_mode(RES)
clock = pygame.time.Clock()

# Fields for current and next grid states
next_field = [[0 for i in range(W)] for j in range(H)]
current_field = [[0 for i in range(W)] for j in range(H)]
current_field = [[1 if i == W // 2 or j == H // 2 else 0 for i in range(W)] for j in range(H)]

# Function to check cell state based on Conway's Game of Life rules
def check_cell(current_field, x, y):
    count = 0
    for j in range(y - 1, y + 2):
        for i in range(x - 1, x + 2):
            if current_field[j][i]:
                count += 1

    if current_field[y][x]:
        count -= 1
        if count == 2 or count == 3:
            return 1
        return 0
    else:
        if count == 3:
            return 1
        return 0

# Class for recording Pygame frames and saving as GIF
class PygameRecord:
    def __init__(self, filename: str, fps: int):
        self.fps = fps
        self.filename = filename
        self.frames = []

    def add_frame(self):
        curr_surface = pygame.display.get_surface()
        x3 = pygame.surfarray.array3d(curr_surface)
        x3 = np.moveaxis(x3, 0, 1)
        array = Image.fromarray(np.uint8(x3))
        self.frames.append(array)

    def save(self):
        self.frames[0].save(
            self.filename,
            save_all=True,
            optimize=False,
            append_images=self.frames[1:],
            loop=0,
            duration=int(1000 / self.fps),
        )

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print(f"An exception of type {exc_type} occurred: {exc_value}")
        self.save()
        return False

# Countdown duration (in seconds)
countdown_seconds = 60  # Change this to the desired countdown time

# Set FPS for recording and game loop
FPS = 30
start_time = pygame.time.get_ticks()  # Record the start time in milliseconds

with PygameRecord("output.gif", FPS) as recorder:
    elapsed_time = 1
    remaining_time = 1
    while remaining_time >= 0:
        elapsed_time = (pygame.time.get_ticks() - start_time) / 1000  # Time in seconds
        remaining_time = countdown_seconds - elapsed_time
        print(f"Time remaining: {int(remaining_time)}s")

        if remaining_time <= 0:
            recorder.save()  # Save GIF when the countdown ends
            print("Countdown finished!")
            pygame.quit()
        
        surface.fill(pygame.Color('black'))
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                recorder.save()
                exit()

        # Draw the grid
        [pygame.draw.line(surface, pygame.Color('darkslategray'), (x, 0), (x, HEIGHT)) for x in range(0, WIDTH, TILE)]
        [pygame.draw.line(surface, pygame.Color('darkslategray'), (0, y), (WIDTH, y)) for y in range(0, HEIGHT, TILE)]

        # Draw cells
        for x in range(1, W - 1):
            for y in range(1, H - 1):
                if current_field[y][x]:
                    pygame.draw.rect(surface, pygame.Color('forestgreen'), (x * TILE + 2, y * TILE + 2, TILE - 2, TILE - 2))
                next_field[y][x] = check_cell(current_field, x, y)

        current_field = deepcopy(next_field)

        # Add a countdown timer display on screen
        font = pygame.font.Font(None, 50)
        text = font.render(f"Time remaining: {int(remaining_time)}s", True, pygame.Color('white'))
        surface.blit(text, (10, 10))

        # Print FPS and add frame to the recorder
        print(clock.get_fps())
        recorder.add_frame()
        pygame.display.flip()
        clock.tick(FPS)


Time remaining: 60s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
0.0
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.77777671813965
Time remaining: 59s
27.027027130126953
Time remaining: 59s
27.027027130126953
Time remaining: 59s
27.027027130126953
Time remaining: 59s
27.027027130126953
Time remaining: 59s
27.027027130126953
Time remaining: 59s
27.027027130126953
Time remaining: 58s
27.027027130126953
Time remaining: 58s
27.027027130126953
Time remaining: 58s
27.027027130126953
Time 

In [4]:
from IPython.display import HTML
HTML('<img src="output.gif">')